In [1]:
import numpy as np

## Load & Convert to Sliding Windows

In [11]:
states = np.load("data/states.npy")
labels = np.load("data/labels.npy")

states = states.reshape((-1, 100, 52))
labels = labels.reshape((-1, 100, 1))

print(states.shape)
print(labels.shape)

(1000, 100, 52)
(1000, 100, 1)


In [12]:
# Create sliding window arrays

original_array = states
original_labels = labels

# Define window size
window_size = 5

# Calculate the number of windows
num_windows = original_array.shape[1] - window_size + 1

# Create a list to store the windows
sliding_windows = []
sliding_labels = []

# Generate sliding windows
for i in range(num_windows-1):
    window = original_array[:, i : i + window_size, :]
    lab = original_labels[:, i + window_size, :]
    sliding_windows.append(window)
    sliding_labels.append(lab)

# Convert the list of windows to a NumPy array
sliding_windows_array = np.array(sliding_windows)
sliding_labels = np.array(sliding_labels)

print("Original Array Shape:", original_array.shape)
print("Sliding Windows Array Shape:", sliding_windows_array.shape)

print("Original Labels Shape:", original_labels.shape)
print("Sliding Windows Labels Shape:", sliding_labels.shape)

Original Array Shape: (1000, 100, 52)
Sliding Windows Array Shape: (95, 1000, 5, 52)
Original Labels Shape: (1000, 100, 1)
Sliding Windows Labels Shape: (95, 1000, 1)


In [14]:
# Collapse into a linear dataset of windows and labels
states_windows = sliding_windows_array.reshape((-1, 5, 52))
labels_windows = sliding_labels.reshape((-1, 1))

print(states_windows.shape)
print(labels_windows.shape)

np.save("data/states_windows.npy", states_windows)
np.save("data/labels_windows.npy", labels_windows)

(95000, 5, 52)
(95000, 1)


## Training Setup

In [15]:
import torch
from controller.ctrl import LSTMModel, LSTMDataset
from torch.utils.data import Dataset, DataLoader, random_split

In [17]:
states_windows = np.load("data/states_windows.npy")
labels_windows = np.load("data/labels_windows.npy")

In [19]:
# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
# Create Dataset and Dataloader
BATCH_SIZE = 128
lstm_dataset = LSTMDataset(states_windows, labels_windows)

train_size = int(0.8 * len(lstm_dataset))
eval_size = len(lstm_dataset) - train_size


train_dataset, eval_dataset = random_split(lstm_dataset, [train_size, eval_size])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [23]:
# Create LSTM Model
INPUT_DIM = 52
HIDDEN_DIM = 100
LAYER_DIM = 2
OUT_DIM = 1
LEARNING_RATE = 1e-3



## Training Stage

In [25]:
NUM_EPOCHS = 5
results = []
for i in range(5):
    print("Trial", i)
    lstm_model = LSTMModel(INPUT_DIM, HIDDEN_DIM, LAYER_DIM, OUT_DIM).to(device)

    # Optimizer
    optimizer = torch.optim.Adam(lstm_model.parameters(), lr=LEARNING_RATE)

    # Loss Function
    loss_fn = torch.nn.BCELoss()

    iter = 0
    iter_res = []
    for epoch in range(NUM_EPOCHS):
        for i, (states, labels) in enumerate(train_loader):
            states = states.to(device)
            labels = labels.to(device)
            
            preds = lstm_model(states)
            loss = loss_fn(preds, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            iter += 1
            iter_res.append(loss.item())
            if iter % 100 == 0:
                # Calculate Loss
                print(f'Epoch: {epoch + 1}/{NUM_EPOCHS}\t Iteration: {iter}\t Loss: {loss.item()}')
    results.append(iter_res)

Trial 0
Epoch: 1/5	 Iteration: 100	 Loss: 0.13921499252319336
Epoch: 1/5	 Iteration: 200	 Loss: 0.11256051063537598
Epoch: 1/5	 Iteration: 300	 Loss: 0.1519027054309845
Epoch: 1/5	 Iteration: 400	 Loss: 0.09588506817817688
Epoch: 1/5	 Iteration: 500	 Loss: 0.0485810711979866
Epoch: 2/5	 Iteration: 600	 Loss: 0.03915582597255707
Epoch: 2/5	 Iteration: 700	 Loss: 0.04482727497816086
Epoch: 2/5	 Iteration: 800	 Loss: 0.0481431670486927
Epoch: 2/5	 Iteration: 900	 Loss: 0.032337017357349396
Epoch: 2/5	 Iteration: 1000	 Loss: 0.07930101454257965
Epoch: 2/5	 Iteration: 1100	 Loss: 0.05403017997741699
Epoch: 3/5	 Iteration: 1200	 Loss: 0.03795018419623375
Epoch: 3/5	 Iteration: 1300	 Loss: 0.049757592380046844
Epoch: 3/5	 Iteration: 1400	 Loss: 0.04945026710629463
Epoch: 3/5	 Iteration: 1500	 Loss: 0.03282295539975166
Epoch: 3/5	 Iteration: 1600	 Loss: 0.019584709778428078
Epoch: 3/5	 Iteration: 1700	 Loss: 0.028952032327651978
Epoch: 4/5	 Iteration: 1800	 Loss: 0.01570698991417885
Epoch: 4/5

In [27]:
np.save("results/lstm_results.npy", results)

## Evaluation

In [10]:
import tqdm

lstm_model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    # progress = tqdm.tqdm(eval_loader, total=len(eval_loader))
    
    # Iterate through test dataset
    for states, labels in eval_loader:
        images = states.to(device)
        labels = labels.to(device)

        outputs = lstm_model(images)
        predicted = (outputs > 0.5).float()
        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    # Print Accuracy
    print(f'Accuracy: {accuracy}')

Accuracy: 98.75263157894737


In [ ]:
# Save models
SAVE_PATH = 'Models/controller/lstm_model.pth'
torch.save(lstm_model.state_dict(), SAVE_PATH)